In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import multiprocessing
from multiprocessing import Pool
from bb_binary import FrameContainer, Repository, load_frame_container
import pandas as pd
import numpy as np
from collections import namedtuple 
import networkx as nx

from pandas import DataFrame, Series

import bib.preprocessing as prep

# Create netwok slices with windowsize for folder

In [ ]:
path = "00_Data/testset_2015_20m/2015082215/"

In [ ]:
ilen = 3
distance = 160
confidence = .95

In [ ]:
# packt immer die 4 Kameradateien zusammen
def get_files(path):
    repo = Repository(path)
    file = list(repo.iter_fnames())
    a = [f.split('/')[-1].split("_")[1] for f in file]
    l = len(a)/4
    npa = np.array(file).reshape(int(l),4)
    return npa

In [ ]:
files = get_files(path)

In [ ]:
pool = multiprocessing.Pool( multiprocessing.cpu_count() )

In [ ]:
tasks_files = []

for e, f in enumerate(files):
    tasks_files.append((e, f, confidence, distance, ilen))

In [ ]:
def generate_timeseries(index, file_list, confidence=.95, distance=160, ilen=3 ):
    
    xmax = 3000
    
    # one df per cam
    dataframes = np.empty(4, dtype=object)

    for i in list(range(4)):
        fc = load_frame_container(file_list[i])
        df = prep.get_dataframe2(fc)
        df = prep.calcIds(df,confidence)
        df = prep.calcIds(df,CONFIDENCE)
        df['frame_idx'] = df['frame_idx'] + index*1024

        camIdx = int(file_list[i].split("/")[-1].split("_")[1])
        dataframes[camIdx] = df
    
    return dataframes

In [ ]:
results = [pool.apply_async( generate_timeseries, t ) for t in tasks]

In [ ]:
dataframes = [DataFrame(), DataFrame(), DataFrame(), DataFrame()]

In [ ]:
for result in results:
    df_list = result.get()
    for e, df in enumerate(df_list):
        dataframes[e] = pd.concat([dataframes[e], df])